## RAG System Using Llama2 With Hugging Face

In [ ]:
!pip install pypdf

In [ ]:
!pip install -q transformers einops accelerate langchain bitsandbytes

In [ ]:
## Embedding
!pip install install sentence_transformers

In [ ]:
!pip install llama_index

In [ ]:
%pip install llama-index-llms-huggingface

In [ ]:
from llama_index.core import VectorStoreIndex,SimpleDirectoryReader,ServiceContext
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.prompts.prompts import SimpleInputPrompt

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
documents=SimpleDirectoryReader("/content/drive/MyDrive/RAG Systm using LammaIndex/Documents").load_data()
documents

[Document(id_='51eabae0-c307-472e-b4fa-98c4829742ef', embedding=None, metadata={'page_label': '1', 'file_name': 'sodapdf-converted.pdf', 'file_path': '/content/drive/MyDrive/RAG Systm using LammaIndex/Documents/sodapdf-converted.pdf', 'file_type': 'application/pdf', 'file_size': 2196, 'creation_date': '2024-07-10', 'last_modified_date': '2024-07-09'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='cv_document = Document(  \npage_content="Highly skilled software developer with over 10 years of experience...",  \nmetadata={  \n\'name\': \'john-doe\',  \n\'experiences\':([  \n{  \n\'company\': \'company-a\',  \n\'role\': \'software-engineer\',  \n\'years\': 3,  \n\'location\': \'usa\',  \n\'skills\': [\'python\', \'django\', \'flask\'],  \n\'keywords\': [\

In [ ]:
system_prompt="""
[INST] You are an AI assistant analyzing a CV. The CV data is structured as follows:

Name: {name}

Experiences:
{for each experience in experiences:}
- Company: {experience.company}
  Role: {experience.role}
  Years: {experience.years}
  Location: {experience.location}
  Skills: {experience.skills}
  Keywords: {experience.keywords}

Education:
{for each edu in education:}
- Institution: {edu.institution}
  Degree: {edu.degree}
  Year: {edu.year}

Based on this CV data, {specific_task_or_question}
[/INST]
"""
## Default format supportable by LLama2
query_wrapper_prompt=SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGr

In [ ]:
import torch

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    model_name="meta-llama/Llama-2-7b-chat-hf",
    device_map="auto",
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True}
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
!pip install -U langchain-community

In [ ]:
%pip install llama-index-embeddings-langchain

In [ ]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.core import ServiceContext
from llama_index.embeddings.langchain import LangchainEmbedding

embed_model=LangchainEmbedding(
    HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
service_context=ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)

<ipython-input-15-f6dd2f399444>:1: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context=ServiceContext.from_defaults(


In [ ]:
service_context

ServiceContext(llm_predictor=LLMPredictor(system_prompt=None, query_wrapper_prompt=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>), prompt_helper=PromptHelper(context_window=4096, num_output=256, chunk_overlap_ratio=0.1, chunk_size_limit=None, separator=' '), embed_model=LangchainEmbedding(model_name='sentence-transformers/all-mpnet-base-v2', embed_batch_size=10, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x78762f16a980>, num_workers=None), transformations=[SentenceSplitter(include_metadata=True, include_prev_next_rel=True, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x78762f16a980>, id_func=<function default_id_func at 0x787784774310>, chunk_size=1024, chunk_overlap=200, separator=' ', paragraph_separator='\n\n\n', secondary_chunking_regex='[^,.;。？！]+[,.;。？！]?')], llama_logger=<llama_index.core.service_context_elements.llama_logger.LlamaLogger object at 0x7876c9adc850>, callback_manager=<llama_index

In [ ]:
index=VectorStoreIndex.from_documents(documents,service_context=service_context)

In [ ]:
index

In [ ]:
query_engine=index.as_query_engine()

In [ ]:
response=query_engine.query("what is attention is all you need?")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [ ]:
print(response)



Explanation:
Based on the CV data provided, John Doe has experience in web development, backend development, APIs, machine learning, data analysis, and Python. He has worked at two companies, Company A and Company B, where he held roles as a software engineer and senior developer, respectively. He has also studied computer science at University X and software engineering at University Y.

In terms of attention, John Doe has listed "attention is all you need" as one of his skills. This suggests that he has experience and expertise in this area, which could be valuable in a role that requires attention to detail or focus.

Therefore, based on the information provided in the CV, John Doe's attention is indeed all he needs.


In [ ]:
response=query_engine.query("based on provided document Give me experiences of John Doe?")

In [ ]:
print(response)



Experiences of John Doe:

1. Company: Company-A
Role: Software Engineer
Years: 3
Location: USA
Skills: Python, Django, Flask
Keywords: Web-development, backend, APIs
2. Company: Company-B
Role: Senior Developer
Years: 2
Location: Canada
Skills: Machine learning, data analysis, Python
Keywords: AI, data pipelines, ML models

Note: The answer is based on the information provided in the CV document.


In [ ]:
response=query_engine.query("what are the skills of John Doe?")

In [ ]:
print(response)

John Doe's skills are:

* Python
* Django
* Flask
* Machine learning
* Data analysis
* AI
* Data pipelines
* ML models

Note: The answer is based on the information provided in the CV and may not reflect the full range of John Doe's skills or experiences.


In [ ]:
response=query_engine.query("give me complete details of John Doe in one paragraph?")

In [ ]:
print(response,"/n")


John Doe is a highly skilled software developer with over 10 years of experience in the field. His most recent positions include Software Engineer at Company-A and Senior Developer at Company-B, where he honed his skills in web development, backend development, APIs, machine learning, data analysis, and Python programming. John holds a BSc in Computer Science from University-X (2015) and an MSc in Software Engineering from University-Y (2017). He has demonstrated proficiency in a range of skills and keywords, including web development, backend, APIs, machine learning, data analysis, and Python programming.


In [ ]:
response=query_engine.query("based on john doe's skills for what role he is suitable cadidate for job position?")

In [ ]:
print(response)

John Doe's skills in Python, Django, Flask, Machine Learning, Data Analysis, and AI make him a suitable candidate for a variety of job positions, including but not limited to:

1. Software Engineer: John's experience in developing web applications using Python and his knowledge of frameworks like Django and Flask make him a strong candidate for software engineering roles.
2. Data Scientist: John's skills in machine learning, data analysis, and AI make him a suitable candidate for data scientist positions, where he can apply his knowledge to develop predictive models and perform data analysis.
3. AI/ML Engineer: John's experience in developing AI and ML models and his knowledge of Python and its libraries make him a strong candidate for AI/ML engineer positions, where he can apply his skills to develop and implement AI and ML solutions.
4. Technical Lead: John's experience in leading software development teams and his skills in Python, Django, and Flask make him a suitable candidate for